In [ ]:
"""
For synchronization between Pesaran Rig (PR) and BISC relay station,
PR generated a psuedorandom bitstream that was captured by both systems.
- BISC relay station captured the bitstream at 33.9kS/s.
- PR had its own clock (precision down to ns, probably some NI component) for sampling
this bitstream at around 1kS/s. Another thing to note about PR is that although the bitstream
itself is captured at ~1kS/s, their time stamps are saved once every ~17 samples.

To recover synchronization, this notebook does the following, in order.
1. PR time stamps are interpolated to 1kS/s
2. PR time stamp and bitstreams are upsampled to 33.9 kS/s
3. Compute time lag between the two systems based on (auto)correlation

Finally, there are time stamps associated with each video frame running at ~60Hz
These time stamps are also captured by the PR clock (hence have the same unit and offset)
4. Based on the time lag between the two systems, and treating t=0 as the beginning of BISC
recording, appropriate offset is applied to these video frame time stamps and saved

Additional Note:
PR's clock and BISC clock are running independently (not synchronized), but this notebook
does not correct for the "clock drift". Effect of drift is negligible since the lengths of
recording are short (less than 2 min). Error due to drift is less than 3 ms. Note that video
frames are captured at 60 Hz (16 ms)

------------------------------

Not really relevant, but ... this is an example of how PR's clock maps to real time.
session 6: 143539
real time - 14:35:46 <=> RIG timestamp 78272463418821

"""

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np
import json
from scipy.io import loadmat

from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *
from utils_motion import RIG_SYNC_HIGH, MOTION_IDX_DICT, read_sync_h5, correlate_custom
from utils_motor_misc import list_files_with_keyword_extension

sys.path.append(UTILS_DIR)
from utils_mp import read_recdata

In [ ]:
def read_pulses(rec_dir, rec_fname, use_tetrode, Ts, sel_invert=True):
    _, pulses, _, _, _ = read_recdata(file_path=f'{rec_dir}/{rec_fname}',
                                               use_tetrode=use_tetrode)

    # print(f'bisc sync length: {len(pulses)*Ts:.1f} sec')
    n = pulses.shape[0]
    bisc_t = np.arange(n)*Ts
    if sel_invert: # hardware configuration
        bisc_sync = -1*pulses + 1 # invert

    return bisc_t, bisc_sync

In [ ]:
""" save plot of sync pulses after synchronization. for sanity check """
sync_save_dir = f'{MOTION_DIR}/sync_img'
if not os.path.exists(sync_save_dir):
    os.makedirs(sync_save_dir)

In [ ]:
session = 3
h5name = f'test.tha.20230919.{session}'
motion_dir = f'{RAW_MOTION_DIR}/{session:03}'
sample_interval, sample_rate, received, rig_sync = read_sync_h5(motion_dir, h5name)

In [ ]:
for session in GOOD_SESSIONS:

    """ Load sync pulses captured by BISC relay station """
    rec_dir = f"{RAW_REC_DIR}/{session:03}"
    if not os.path.isdir(rec_dir):
        continue
    print(f'processing: session {session}')

    assert len(os.listdir(rec_dir)) == 1
    rec_fname =  os.listdir(rec_dir)[0]
    bisc_t, bisc_sync = read_pulses(rec_dir, rec_fname, USE_TETRODE, TS)

    """ Load sync pulses & their time stamps captured by PR """
    motion_dir = f'{RAW_MOTION_DIR}/{session:03}'
    h5name = f'test.tha.20230919.{session}'

    # received: Nx2 array. column 0: timestamp in ns, column 1: cumulative number of data points collected
    # data: raw sample data (sync low: 0V, high: 5V)
    sample_interval, sample_rate, received, rig_sync = read_sync_h5(motion_dir, h5name)
    assert received[-1, 1] == rig_sync.shape[0] # total number of sync pulse data points

    sample_interval = sample_interval*1e-9 # convert nanosec to sec

    # convert to binary (sync does not need unrolling)
    rig_sync = np.squeeze(rig_sync/RIG_SYNC_HIGH)
    rig_sync = rig_sync.astype(np.int64) # not sure why 64-bit was allocated.. maybe for correlation computing later on?
    rig_t = np.zeros_like(rig_sync)

    """ 1. PR time stamps are interpolated to 1kS/s """
    rig_t_coarse = received[:, 0]
    # number of samples per timestamp
    npb = np.concatenate(([received[0,1]], np.diff(received[:,1])), dtype=int)

    idx = 0
    for n, t0, t1 in zip(npb, rig_t_coarse[:-1], rig_t_coarse[1:]):
        rig_t[idx:idx + n] = t0 + ((t1-t0)/n)*np.linspace(0, n-1, n)
        idx += n
    n = npb[-1]
    rig_t[idx:idx + n] = t1 + sample_interval*np.linspace(0, n-1, n)

    # force to start at 0, and convert to sec
    rig_t0 = rig_t[0].astype(np.int64)
    rig_t -= rig_t0
    rig_t = rig_t*1e-9
    # rig_t = (rig_t - rig_t0)*1e-9

    """ 2. PR time stamp and bitstreams are upsampled to 33.9 kS/s """
    n = int(rig_t[-1]*FS)
    upsamp_t = np.arange(n)*TS
    upsamp_sync = np.interp(upsamp_t, rig_t, rig_sync)

    """ 3. Compute time lag between the two systems based on (auto)correlation """
    # Run Coarse Cross Correlation on Decimated Data
    kdf = 100 # decimation factor
    # correlate(a, v, mode='full). first element of the output: a[0]*v[-1]
    coarse_corr = np.correlate(bisc_sync[::kdf]-0.5, upsamp_sync[::kdf]-0.5, mode='full')

    i_coarse_shift = kdf*(np.argmax(coarse_corr))
    t_coarse_shift = (i_coarse_shift - len(upsamp_t)) * TS

    # Run Fine Cross Correlation
    fine_corr = correlate_custom(bisc_sync-0.5, upsamp_sync-0.5,
                                  i_coarse_shift-kdf, i_coarse_shift+kdf)

    i_shift = (np.argmax(fine_corr))
    # t_shift is the time lag between the two systems,
    # from "the first sync pulse captured on PR" to "the first sync pulse captured on BISC"
    t_shift = (i_shift - len(upsamp_t)) * TS

    """ Save Plot of Time Lag Corrected Syncs """
    fig, ax = plt.subplots(3, 1, figsize=(10, 6), sharex=True)

    ax[0].set_title(f'Session {session:03}\n Rig, Before Upsample')
    ax[0].plot(rig_t + t_shift, rig_sync)

    ax[1].set_title('Rig, Upsampled')
    ax[1].plot(upsamp_t + t_shift, upsamp_sync)

    ax[2].set_title('BISC sync')
    ax[2].plot(bisc_t, bisc_sync)

    fig.savefig(f'{sync_save_dir}/{session:03}.png', bbox_inches='tight')
    plt.close(fig)

    """ Load Motion Data """
    # timestamps are in cam3 json. likelihood and position are in *.mat file
    # load cam3 time
    cam3_json_fname = f'troopa_230919_{session:03}_cam3.json'
    with open(f'{motion_dir}/{cam3_json_fname}', 'r') as file:
        cam3_t = np.array(json.load(file))

    # Load motion v3 time point and correct it
    motion_t = loadmat(f'{motion_dir}/rec_{session:03}_timestamp.mat')
    motion_t = np.squeeze(motion_t['timestamps_rec'])

    """ 4. Based on the time lag between the two systems, and treating t=0 as the beginning
    of BISC recording, appropriate offset is applied to these time stamps and saved"""
    # force first sample to be 0
    motion_t -= motion_t[0]
    # apply correction against the "rig time"
    motion_t += (cam3_t[0] - rig_t0)*1e-9
    # apply correction against BISC time
    motion_t += t_shift 

    """ save """
    save_dir = f'{MOTION_DIR}/{session:03}'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    keys = [key for key in MOTION_IDX_DICT.keys() if key.startswith(f'{session:003}')]

    # save time points over the whole session
    np.save(f'{save_dir}/motion_t_{session:003}.npy', motion_t)
    
    # for key in keys:
    #     (idx0, idx1) = MOTION_IDX_DICT[key]
    #     if idx1 == -1: idx1 = len(motion_t)
    
    #     np.save(f'{save_dir}/pos_t_session_{key}.npy', motion_t[idx0:idx1])
    #     print(key)

In [ ]:
### write assertion checks..!
for session in GOOD_SESSIONS:
    keys = [key for key in MOTION_IDX_DICT.keys() if key.startswith(f'{session:003}')]

    dir0 = f'./motion_v3_postprocess_bak/{session:003}'
    dir1 = f'./motion_v3_postprocess/{session:003}'
    
    t0 = np.load(f'{dir0}/motion_t.npy')
    t1 = np.load(f'{dir1}/motion_t_{session:003}.npy')
    assert np.array_equal(t0, t1)

    for key in keys:
        pass
        # t_v3p0 = np.load(f'{dir0}/motion_t_v3p0_session_{key}.npy')
        # t_v3p1 = np.load(f'{dir0}/motion_t_v3p1_session_{key}.npy')
        # print(key)
        # assert np.array_equal(t_v3p0, t_v3p1)